In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import library

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline


**Import Data**

In [ ]:
df= pd.read_csv('/kaggle/input/covid19-vaccine-tweets/file.csv')

df.head()

**Only Englsih language tweet**

In [ ]:
df = df[df['language']=='en']

In [ ]:
data = df[['tweet']]
data.head()

In [ ]:
data= data.reset_index(drop=True)
data['process_tweet'] = data['tweet'].copy()
data.head()

**Import library**

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import transformers as ppb

from tqdm import tqdm_notebook as tqdm
import random
import matplotlib.pyplot as plt
import warnings


#import library

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder   ###########
from sklearn.metrics import confusion_matrix , classification_report , accuracy_score
from sklearn.manifold import TSNE ######
from sklearn.feature_extraction.text import TfidfVectorizer #############

from keras.preprocessing.text import Tokenizer         #######
from keras.preprocessing.sequence import pad_sequences #######
from keras.models import Sequential
from keras.layers import Dropout , Conv1D ,MaxPool1D,Activation , Dense , Flatten , Embedding , LSTM ####

from keras import utils
from keras.callbacks import ReduceLROnPlateau , EarlyStopping


#nltk

import nltk
from nltk.corpus import stopwords ########
from nltk.stem import SnowballStemmer ############


#word to vec
import gensim

import re #####
import os
from collections import Counter #######
import logging ###
import time
import pickle ######
import itertools ######


from textblob import TextBlob # TextBlob - Python library for processing textual data





**Data procesing**

In [ ]:
def clean_data(txt):
  txt = txt.lower()  # lowercase
  txt = re.sub(r'@[A-Za-z0-9_]+' , '' , txt)   #remove mentions
  txt = re.sub(r'#' , '' , txt) #remove hashtags
  txt = re.sub(r'RT : ','' , txt) # remove retweets
  txt = re.sub(r'https?:\/\/[A-Za-z0-9\./\/]+' , '' , txt) #removes url
  txt = re.sub('\[.*?\]' , '' , txt) #remove square brackets
  txt = re.sub(r'[^\w\s]' , '' , txt) #remove puntuations
  txt = re.sub('\w*\d\w*' , '' ,txt) #removes words containig numbers
  txt = re.sub('\n' , '' ,txt) #remove new lines

  return txt


In [ ]:
data['process_tweet'] = data['process_tweet'].apply(clean_data)

**Remove Stopwords**

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')

def remove_stopwords(txt):
  rmv_stpwords = [i for i in txt.split() if i not in stopwords]
  rmv_stpwords_join = ' '.join(rmv_stpwords)
  return rmv_stpwords_join


In [ ]:
data['process_tweet'] = data['process_tweet'].apply(remove_stopwords)

data.head()#after removing stopwords

**PorterStemmer**

In [ ]:
from nltk.stem import PorterStemmer
st = PorterStemmer()

In [ ]:
def stemming(txt):
  txt =[st.stem(word) for word in txt.split()]
  txt_join = ' '.join(txt)
  return txt_join

In [ ]:
data['process_tweet'] = data['process_tweet'].apply(stemming)
data.head()

**Using TextBlob calculate the polarity**

In [ ]:
def get_text_polarity(txt):
  return TextBlob(txt).sentiment.polarity

In [ ]:
data['Polarity'] = data['process_tweet'].apply(get_text_polarity)

**Labeling using the polarity score**

In [ ]:
#Labeling
def get_text_analysis(i):
  if (i<-0.5):
    return 'Strongly Negative'
  elif ((i<0 ) and (i >= -0.5)):
    return 'Negative'
  elif (i == 0):
    return 'Neutral'
  elif (i>0 and i<=0.5):
    return 'Positive'
  else:
    return 'Strongly Positive'


In [ ]:
data['Sentiment'] = data['Polarity'].apply(get_text_analysis)

In [ ]:
#for deep learning we need to only 2 colums(sentiment and process tweet)
data = data[['Sentiment','process_tweet' ]]

possible_labels = data.Sentiment.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels): 
#The enumerate() function takes a collection (e.g. a tuple) and returns it as an enumerate object.
    label_dict[possible_label] = index

data['label'] = data.Sentiment.replace(label_dict)


print(label_dict)

data.head()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
ax = data['Sentiment'].value_counts(sort=False).plot(kind='barh')
ax.set_xlabel('Number of Samples in training Set')
ax.set_ylabel('Label')

**Total Unique words after pre processing**

In [ ]:
unique_words = set(data['process_tweet'])
count = 0
for word in unique_words:
  count += 1
print(count)

**Tokenization**

In [ ]:
#tokenization
tokenizer = Tokenizer(num_words=163967, split=' ')
#num_words: the maximum number of words to keep, based on word frequency. Only the most common num_words-1 words will be kept.


tokenizer.fit_on_texts(data['process_tweet'].values)

X = tokenizer.texts_to_sequences(data['process_tweet'].values)

In [ ]:
X[:3] #before padding

**Padding**

In [ ]:
#padding to make all text vector to same length

X = pad_sequences(X)

X[:3] #after padding

**RNN Model**

In [ ]:
#creating models

model= Sequential()
model.add(Embedding(163967, 256 , input_length=X.shape[1]))
model.add(Dropout(0.3))


model.add(LSTM(128 , return_sequences=True , dropout=0.3 , recurrent_dropout=0.3))
model.add(LSTM(128,dropout=0.3 , recurrent_dropout=0.3))


model.add(Dense(5 , activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam' , loss = 'categorical_crossentropy' , metrics=['accuracy'])

In [ ]:
#one hot encoding

y = pd.get_dummies(data['Sentiment']).values

[print(data['Sentiment'][i] ,y[i]) for i in range(0,5)]

**Data split**

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(X , y , test_size = 0.2 ,random_state = 22)

In [ ]:
#trannig model
batch_size = 128
epochs = 5

**Tranning the model**

In [ ]:
history = model.fit(x_train , y_train ,
          epochs = epochs,
          batch_size = batch_size,
          validation_split=0.1,
          verbose=1)

In [ ]:
%%time
score = model.evaluate(x_test, y_test , batch_size = batch_size)
print()
print("ACCURACY:",score[1])
print("LOSS:",score[0])

In [ ]:
predictions = model.predict(x_test)
pred1 = np.argmax(predictions , axis=1)
pred1[:10]

In [ ]:
y_test = np.argmax(y_test , axis=1)


In [ ]:
print(confusion_matrix(y_test , pred1))

**Classification Report**

In [ ]:
print(classification_report(y_test , pred1))

**Accuracy**

In [ ]:
accuracy_score(y_test , pred1)

In [ ]:
epochs = [i for i in range(5)]
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
fig.set_size_inches(30,10)


ax[0].plot(epochs , train_acc , 'go-' , label = 'Training Accuracy')
ax[0].plot(epochs , val_acc , 'ro-' , label = 'Validation Accuracy')
ax[0].set_title('Training & Validation Accuracy')

ax[0].legend()
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")


ax[1].plot(epochs , train_loss , 'g-o' , label = 'Training Loss')
ax[1].plot(epochs , val_loss , 'r-o' , label = 'Validation Loss')
ax[1].set_title('Testing Accuracy & Loss')
ax[1].legend()
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Training & Validation Loss")
plt.show()